<a href="https://colab.research.google.com/gist/peterlafferty/0c03ac8dcf661b72074a75f0a2eaad6a/copy-of-copy-of-colab_with_google_sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#  [Causal Impact] Python Colab with Google Sheets

Causal Impact can be used to estimate the impact an event has on a key performance indicator (KPI) using synthetic control groups that are correlated to the KPI but not impacted by the event.

This Colab runs a Causal Impact analysis using a Google Sheet for inputs.

## Steps for Running
* Define KPI to measure
* Choose control groups that are correlated but not impacted.
* Collect + populate the data.
* Run the analysis.
* Understand the output.

## Picking Control Groups

* Unaffected predictors: The covariates (control) are unaffected by the intervention.
* Stable relationship: The relationship between the covariates (control) and the outcome variable (response) would have remained stable under no treatment.

If the target KPI can be split by location then the markets where the event isn't used can be used as control groups. For example conversions in UK around an Diwali related advert is run can be compared to conversions in France where the advert wasn't run. Even though the control group is the same as the KPI it can't be used as a covariate as it isn't impacted by the advert.

Control groups should be related to the outcome but not changed by the event. To check the incrementallity effect on conversions from a branded campaign on YouTube the covariates to use could be clicks, sessions, new users from site analytics.


## Use Cases For Causal Impact
* Check the impact of a branding campaign.
* Validate account restucturing on campaigns.
* Measure the impact of a drop in spend.
* Prove the incremental value of running additional campaign types.
* Post analysis of an unplanned event.

## More Information
Visit the [tfp-causalimpact](https://github.com/google/tfp-causalimpact) library on GitHub for more details on the implementation and algorithm.

See also `Inferring causal impact using Bayesian structural time-series models.` Kay H. Brodersen, Fabian Gallusser, Jim Koehler, Nicolas Remy, Steven L. Scott. Annals of Applied Statistics, vol. 9 (2015), pp. 247-274. https://research.google/pubs/pub41854/




## Google Sheet Template Configuration

Make a copy of this Google Sheet and use the copy in the input below
 https://docs.google.com/spreadsheets/d/1VgA7uMxYF6tsnBwgJqpj4oaLAqYPQlb8pBI0VIDGel4/edit#gid=0


### Valid Formatting

 * Apart from the first column the headings can be changed.
 * The first column is the date of the observation and must be named `date`.
     * The date format must be YYYY-MM-DD.
     * In this Colab the dates are daily values. Not weekly.
     * Order the rows chronologically from oldest to newest.
 * The second column is the key performance indicator (KPI) to analyse.
 * The third column and after are covariates aka control groups.
 * Replace or estimate empty or null values.
 * Numbers must be in the format 1234.56. If there are commas, it will not convert properly (e.g. 1,234.56)


## Configure The Analysis

In [ ]:
# Google Sheet for data streams
google_sheets_url = 'https://docs.google.com/spreadsheets/d/1VgA7uMxYF6tsnBwgJqpj4oaLAqYPQlb8pBI0VIDGel4/edit#gid=0'#@param

#date of event
event_date = '2019-10-07'#@param {type:"date"}

#How many days before intervention will be analyzed
days_before_event = 90#@param {type:"number"}

#how many days post intervention wou want to analyze (including the event date)
days_after_event = 7#@param {type:"number"}

#Number of series used as counterfactuals (0 to many, each in a column of the sheet)
number_of_covariates = 4#@param {type:"number"}

!pip install tfp-causalimpact itables
import pandas as pd
import numpy as np
import causalimpact
from IPython.display import display, Markdown
from itables import show

#convert input to DateTime
event_date = pd.to_datetime(event_date, format='%Y-%m-%d', errors='ignore')

print("Causal Impact version: ", causalimpact.__version__)

## Load Data From Google Sheets

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open_by_url(google_sheets_url).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

df = pd.DataFrame.from_records(rows)

#get column names from first row
df.columns = df.iloc[0]

#drop row with column names
df = df.drop(df.index[0])

#reset indexes
df = df.reset_index(drop=True)

#convert numeric data to numeric types
df = df.apply(pd.to_numeric, errors='ignore')

#convert dates to dates type
df.date = pd.to_datetime(df.date, format='%Y-%m-%d', errors='ignore')



assert event_date in df.date.values, f"The Google Sheet doesn't contain the input date: \"{event_date.date()}\"."

event_index = df.loc[df.date==event_date].index[0]-1;

#explore data
display(Markdown("###Explore the imported data"))
show(df);

display(Markdown("###Validate everything has a value"))
display(Markdown("If any of the values are False there is missing data in that column."))
display(df.notnull().all())


display(Markdown("###Visually compare the covariates"))
normalized_df = df.iloc[:,0:-1].apply(lambda x: (x-x.mean())/ x.std(), axis=0)
normalized_df.plot(x='date', figsize=(12, 5));


## Optional Input Validation

In [ ]:
df.corr(method='pearson', numeric_only=False)

df.describe()

## Run The Analysis

In [ ]:
pre_period = [int(max(0, event_index-days_before_event)), int(event_index)]
post_period = [int(event_index + 1), int(event_index + days_after_event)]

impact = causalimpact.fit_causalimpact(
    data=df[df.columns[1:number_of_covariates + 2]],
    pre_period=pre_period,
    post_period=post_period)

display(Markdown("##Summary report"))
print(causalimpact.summary(impact, output_format='summary'))

display(Markdown("##Plotted results"))
display(causalimpact.plot(impact, static_plot=False, chart_width=800))

display(Markdown("##Detailed Report"))
print(causalimpact.summary(impact, output_format='report'))
